In [ ]:
!pip install pymupdf
!pip install sentence-transformers


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import fitz  # PyMuPDF

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def preprocess_text_modified(text):

    # To convert bullet points into full stops if they follow a semi-colon or start of the text
    text = re.sub(r'(?<=;)\s*●\s*', '. ', text)
    text = re.sub(r'^●\s*', '. ', text, flags=re.MULTILINE)

    # To keep numeric references and legal terminologies intact - customize as necessary
    pattern_to_keep = re.compile(r'\b17Ad-22\(e\)\(7\)\b|\bCCP\b', re.IGNORECASE)
    kept_terms = pattern_to_keep.findall(text)

    # To convert text to lowercase, except for kept_terms
    text = text.lower()

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    processed_sentences = []

    for sentence in sentences:
        # Remove punctuation while preserving intra-word dashes
        sentence = re.sub(r'(?<!\w)-|-(?!\w)', ' ', sentence)  # Replace dashes not within words
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))

        # Tokenization
        tokens = word_tokenize(sentence)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in tokens if word not in stop_words]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

        processed_sentences.append(' '.join(lemmatized_tokens))

    processed_text = '. '.join(processed_sentences)

    # Reinsert numeric references and legal terminologies with original casing
    for term in kept_terms:
        term_lowercase = term.lower()
        processed_text = re.sub(re.escape(term_lowercase), term, processed_text)

    return processed_text


pdf_path1 = '/content/rule 7 doc 1.pdf'
pdf_path2 = '/content/rule 19 doc 1.pdf'

# Extract and preprocess text
text1 = extract_text_from_pdf(pdf_path1)
text2 = extract_text_from_pdf(pdf_path2)

preprocessed_text1 = preprocess_text_modified(text1)
preprocessed_text2 = preprocess_text_modified(text2)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
preprocessed_text1

'rule 17ad22e7 liquidity risk proposed rule proposed rule 17ad22e7 would require covered clearing agency establish implement maintain enforce written policy procedure reasonably designed effectively measure monitor manage liquidity risk arises borne meeting minimum ten requirement specified rule. proposed rule 17ad22e7i would require covered clearing agency ’ policy procedure reasonably designed ensure maintains sufficient liquid resource relevant currency effect sameday appropriate intraday multiday settlement payment obligation high degree confidence wide range potential stress scenario includes default participant family would generate largest aggregate payment obligation extreme plausible market condition. proposed rule 17ad22e7ii would require covered clearing agency establish implement maintain enforce written policy procedure reasonably designed ensure hold qualifying liquid resource sufficient meet minimum liquidity resource requirement relevant currency covered clearing agency

# **BERT**

In [27]:
import torch
from transformers import BertModel, BertTokenizer
from sentence_transformers import util
import numpy as np

# Initialize the tokenizer and model from Hugging Face Transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the texts as PyTorch tensors
inputs1 = tokenizer(preprocessed_text1, return_tensors="pt", padding=True, truncation=True)
inputs2 = tokenizer(preprocessed_text2, return_tensors="pt", padding=True, truncation=True)

# Obtain embeddings
with torch.no_grad():  # No need to compute gradients
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

    # Use the mean of the last layer hidden states as sentence vector representation
    sentence_embedding1 = outputs1.last_hidden_state.mean(dim=1)
    sentence_embedding2 = outputs2.last_hidden_state.mean(dim=1)

# Calculate the cosine similarity between the embeddings using util.pytorch_cos_sim
similarity = util.pytorch_cos_sim(sentence_embedding1, sentence_embedding2)

print(f"cosine similarity is {similarity.item()}")


cosine similarity is 0.9338878393173218


# **SBERT**

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

# Split texts into lists of sentences.
sentences1 = preprocessed_text1.split('. ')
sentences2 = preprocessed_text2.split('. ')

# Encode sentences to get their embeddings
embedding1 = model.encode(sentences1, convert_to_tensor=True)
embedding2 = model.encode(sentences2, convert_to_tensor=True)

# Calculate pairwise cosine similarities between all sentences in both sets
similarity_matrix = util.pytorch_cos_sim(embedding1, embedding2)

# Define a similarity threshold for "high similarity"
similarity_threshold = 0.7

# Extract the sentences and scores for pairs above the threshold
high_similarity_pairs = [{
    'Sentences from rule 7': sentences1[i],
    'Sentences from rule 19': sentences2[j],
    'Cosine Similarity Score': similarity_matrix[i, j].item()
} for i in range(similarity_matrix.size(0)) for j in range(similarity_matrix.size(1)) if similarity_matrix[i, j] > similarity_threshold]

high_similarity_df = pd.DataFrame(high_similarity_pairs)

high_similarity_df.sort_values(by='Cosine Similarity Score', ascending=False, inplace=True)


In [26]:
high_similarity_df.to_csv('Similar_sentences_file1.csv', index=False)


In [24]:
high_similarity_df

,Sentences from rule 7,Sentences from rule 19,Cosine Similarity Score
39,commission also adopting rule 17ad22e7 modific...,final rule commission adopting rule 17ad22e19 ...,0.849316
18,proposed rule 17ad22e7vii would require covere...,addition proposed rule 17ad22e19 would require...,0.825034
44,commission recognizes may number way address c...,commission recognizes may number way address c...,0.824350
11,proposed rule 17ad22e7v would require covered ...,addition proposed rule 17ad22e19 would require...,0.816881
5,proposed rule 17ad22e7ii would require covered...,addition proposed rule 17ad22e19 would require...,0.798960
14,proposed rule 17ad22e7vid would also require c...,addition proposed rule 17ad22e19 would require...,0.797117
28,one commenter stated proposed rule 17ad22e7 pr...,light availability tool described commission b...,0.786520
8,proposed rule 17ad22e7iv would require covered...,addition proposed rule 17ad22e19 would require...,0.786438
1,rule 17ad22e7 liquidity risk proposed rule pro...,addition proposed rule 17ad22e19 would require...,0.779790
4,proposed rule 17ad22e7ii would require covered...,rule 17ad22e19 tiered participation arrangemen...,0.768527


# Calculating the overall cosine similarity between both rules using SBERT

In [ ]:
# Calculate the average embedding for each document
avg_embedding1 = torch.mean(embedding1, dim=0)
avg_embedding2 = torch.mean(embedding2, dim=0)

# Calculate the cosine similarity between the average embeddings of the two documents
overall_similarity = util.pytorch_cos_sim(avg_embedding1, avg_embedding2)

print("Overall similarity between the two documents:", overall_similarity.item())

Overall similarity between the two documents: 0.8473458886146545
